In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
# GPS PRN Extraction
codesFile = scipy.io.loadmat("randomData/codes_L1CA.mat")
codes = np.array(codesFile['codes_L1CA'])

### Q5
Write a MATLAB/ Python/C/C++ program to implement serial search/parallel code phase search 
acquisition algorithm. Identify the satellites (PRN IDs), carrier frequency, and code phase using the
acquisition algorithm in the data file provided in the Google drive

### Sol.
- we know the signal received at the antenna is:


$s(t) = \sqrt{2P}D(t-\tau)C(t-\tau)cos((w_L+w_D)t+\theta)$
where : 
    - P is power
    - $\tau$ is delay ($\tau = \tau_{int} + \tau_{frac}$)
    - $w_L$ is L-5 band frequency and $w_D$ is doppler frequency

- After RF front-end processing we get:

$ I = \sqrt{2P_1}D(t-\tau)C(t-\tau)cos( (w_D)t+\theta)$


$ Q = \sqrt{2P_1}D(t-\tau)C(t-\tau)sin( (w_D)t+\theta)$

where:
    - I is Inphase samples
    - Q is Quadrature samples

**Note** : slight abuse of notation is used as we defined C for discrete values while here we are considering it as continous, but this discrepancy will be resolved in program as we have input data as discrete samples

#### Now,
for serial search we need to get 3 parameters 

- $i$ the satellite number
- $\tau_{frac} \in [0,1022]$
- $w_D \in [-20k Hz : 500 Hz : 20k Hz]$

#### Steps

- Firstly, we will rewrite signal as $S = I + Qj = \sqrt{2P_1}D(t-\tau)C(t-\tau)e^{jw_dt}$

- Then, we will multiply $S$ by $C(t-\hat\tau)e^{-j\hat w_dt}$

- We will get:

$ Corr = \sqrt{2P_1}D(t-\tau)C(t-\tau)\oplus C_i(t-\hat\tau)e^{j(w_d-\hat w_d)t}$

it will attain it's maximum when :

$w_d-\hat w_d = 0$


$\tau = \hat\tau$

$i = $ satellite number of incoming signal

In [2]:
# Assumptions: 
#   - Sampling frequency is Fs which is a multiple of 1.023 MHz
#   - Fs = n*1.023 MHz

def modPRN(PRN,n,tau):
    """
    function to get modified PRN for some sampling frequency & delay
    Inputs: 
        - PRN is the PRN code for single satellite with 1023 length
        - n = SamplingFrequency/1.023e6 
    Output:
        - PRN code for 1 millisecond with length = 1023*n
        - This PRN code is delayed by tau chips
    """

    # return np.roll(np.repeat(PRN,n),-tau)
    return np.repeat(np.roll(PRN,-tau),n)

In [3]:
# Checking modPRN
someArrForModPRN = np.array([1,2,3,4,5])
modPRN(someArrForModPRN,3,4)

array([5, 5, 5, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4])

In [4]:
wds = np.linspace(-20000,20000,41)*2*np.pi
def serialSearch(I, Q, knownPRNs, Fs, wds=wds):
    rcvdSignal = np.array(I+Q*1j)
    n = int(Fs/1.023e6)
    t = np.linspace(0,1,n*1023)*1e-3
    totalSatellites = knownPRNs.shape[1]
    corr = np.zeros((totalSatellites,len(wds),1023*n))
    for x in range(0,totalSatellites):
        for y in range(0,len(wds)):
            for z in range(0,1023*n):
                generatedSignal = modPRN(knownPRNs[:,x],n,z)*np.array(np.exp(-1j*wds[y]*t))
                corr[x,y,z] = np.abs(np.sum(generatedSignal*rcvdSignal))
    return corr




In [5]:
t = np.linspace(0,1,1023)*1e-3

In [6]:
sirPRNI = modPRN(codes[:,20],1,82)*np.cos(2*np.pi*675*t)+np.random.normal(0,4,1023)
sirPRNQ = modPRN(codes[:,20],1,82)*np.sin(2*np.pi*675*t)+np.random.normal(0,4,1023)

In [7]:
corr = serialSearch(sirPRNI,sirPRNQ,codes,1.023e6)

In [8]:
satNum,wdstar,taustar = np.unravel_index(corr.argmax(), corr.shape) 

In [9]:
satNum

20

In [10]:
wds[wdstar]/(2*np.pi)

999.9999999999999

In [11]:
taustar

82